In [2]:
import pandas as pd
import requests
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result = requests.get(url)
print(url)
print(result.status_code)
print(result.headers)

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
200
{'Date': 'Mon, 30 Mar 2020 02:05:03 GMT', 'Content-Type': 'text/html; charset=UTF-8', 'Server': 'mw1321.eqiad.wmnet', 'X-Powered-By': 'PHP/7.2.26-1+0~20191218.33+debian9~1.gbpb5a340+wmf1', 'X-Content-Type-Options': 'nosniff', 'P3P': 'CP="See https://en.wikipedia.org/wiki/Special:CentralAutoLogin/P3P for more info."', 'Content-language': 'en', 'Vary': 'Accept-Encoding,Cookie,Authorization', 'Last-Modified': 'Mon, 30 Mar 2020 02:01:19 GMT', 'Backend-Timing': 'D=122199 t=1585533902960773', 'X-ATS-Timestamp': '1585534141', 'Content-Encoding': 'gzip', 'X-Varnish': '847908558 761488347', 'Age': '5225', 'X-Cache': 'cp1081 hit, cp1081 hit/13', 'X-Cache-Status': 'hit-front', 'Server-Timing': 'cache;desc="hit-front"', 'Strict-Transport-Security': 'max-age=106384710; includeSubDomains; preload', 'Set-Cookie': 'WMF-Last-Access=30-Mar-2020;Path=/;HttpOnly;secure;Expires=Fri, 01 May 2020 00:00:00 GMT, WMF-Last-Access-Global=30-Mar-2

USING THE BEAUTIFULSOUP. WE WILL SAVE ALL THE HTML CONTENT IN result.content to soup VARIABLE ADN THEN WE WILL DO THE ASSIGNED TASKS 

In [3]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(result.content, 'html.parser')
table = soup.find('table')
trs = table.find_all('tr')
rows = []
for tr in trs:
    i = tr.find_all('td')
    if i:
        rows.append(i)
        
lst = []
for row in rows:
    postalcode = row[0].text.rstrip()
    borough = row[1].text.rstrip()
    neighborhood = row[2].text.rstrip()
    if borough != 'Not assigned':
        if neighborhood == 'Not assigned':
            neighborhood = borough
        lst.append([postalcode, borough, neighborhood])

# lst

In [4]:
cols = ['PostalCode', 'Borough', 'Neighborhood']
df = pd.DataFrame(lst, columns=cols)
print(df.shape)

(103, 3)


In [6]:
f = df.groupby('PostalCode').agg(
    {
        'Borough':'first', 
        'Neighborhood': ', '.join,}
    ).reset_index()

In [7]:
df.shape

(103, 3)

In [8]:
!wget -O GeoCord.csv http://cocl.us/Geospatial_data/

--2020-03-30 03:57:12--  http://cocl.us/Geospatial_data/
Resolving cocl.us (cocl.us)... 169.48.113.194, 158.85.108.83, 158.85.108.86
Connecting to cocl.us (cocl.us)|169.48.113.194|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data/ [following]
--2020-03-30 03:57:13--  https://cocl.us/Geospatial_data/
Connecting to cocl.us (cocl.us)|169.48.113.194|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-03-30 03:57:14--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.27.197, 107.152.26.197
Connecting to ibm.box.com (ibm.box.com)|107.152.27.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-03-30 03:57:14--  https

In [9]:
df_cordinates = pd.read_csv('GeoCord.csv') # Read the csv data
df_cordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
import numpy as np
df['Latitude'] = np.nan
df['Longitude'] = np.nan

# For each postcode in df, find corresponding coordinates in df_cord and assign it to df
for idx in df.index:
    cordinates_idx = df_cordinates['Postal Code'] == df.loc[idx, 'PostalCode']
    df.at[idx, 'Latitude'] = df_cordinates.loc[cordinates_idx, 'Latitude'].values
    df.at[idx, 'Longitude'] = df_cordinates.loc[cordinates_idx, 'Longitude'].values

# Display the results
df.head(20)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,Parkview Hill / Woodbine Gardens,43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
